In [1]:
import os, glob, re, cv2
import numpy as np

In [2]:
import os, glob, cv2

CROPS_DIR = "crops"

# store tuples: (filename, bgr_image)
images = []
for ext in ("*.jpg", "*.jpeg", "*.png"):
    for path in glob.glob(os.path.join(CROPS_DIR, ext)):
        bgr = cv2.imread(path)
        if bgr is None:
            continue
        images.append((os.path.basename(path), bgr))

print(f"Loaded {len(images)} images")


Loaded 5 images


In [3]:
import re
from fast_plate_ocr import LicensePlateRecognizer

ocr = LicensePlateRecognizer("cct-xs-v1-global-model")

PLATE_RE  = re.compile(r'^([A-Z]{1,2})(\d{1,4})([A-Z]{1,3})$')
DIGIT_FIX = str.maketrans({'O':'0','I':'1','Z':'2','S':'5','B':'8','G':'6','Q':'0'})

def normalize(raw):
    if not raw:
        return None
    canon = re.sub(r'[^A-Za-z0-9]', '', raw).upper()
    m = PLATE_RE.match(canon)
    if not m:
        return None
    prefix, digits, suffix = m.groups()
    digits = digits.translate(DIGIT_FIX)
    return f"{prefix} {digits} {suffix}"

plate_result = {}  # fname -> normalized/raw plate string

for fname, bgr in images:
    texts = ocr.run(bgr) or []
    raw = texts[0] if texts else ""
    plate = normalize(raw) or raw or "-"
    plate_result[fname] = plate
    print(f"{fname} -> Plate: {plate}")


1_crop0.jpg -> Plate: B 1970 SSW
2_crop0.jpg -> Plate: B 1037 SPW
3_crop0.jpg -> Plate: B 1308 RFI
4_crop0.jpg -> Plate: B 188 BA
5_crop0.jpg -> Plate: CD 7528 A


In [4]:
import re, cv2
import easyocr

# MM-YY with flexible separators
EXPIRY_RE = re.compile(r'(0[1-9]|1[0-2])\s*[-./·:_,| ]\s*([0-9]{2})')

# Reader (set gpu=False if no CUDA)
reader = easyocr.Reader(['en'], gpu=True)
EASYOCR_CFGS = [
    dict(detail=0, paragraph=True, contrast_ths=0.1, adjust_contrast=0.7, text_threshold=0.4, low_text=0.3),
    dict(detail=0, paragraph=True, contrast_ths=0.05, adjust_contrast=0.9, text_threshold=0.3, low_text=0.2),
]

def crop_band_candidates(bgr):
    """Yield several bottom-band crops + small vertical slide to catch misplacement."""
    h, w = bgr.shape[:2]
    for start in (0.55, 0.60, 0.65, 0.70):
        y1 = int(h * start)
        y2 = h
        pad = max(2, h // 50)
        y1 = max(0, y1 - pad)
        band = bgr[y1:y2, :]
        if band.size > 0:
            yield band
            slide = max(2, int(0.05 * h))
            y1s = max(0, y1 - slide)
            y2s = min(h, y2 - slide)
            if y2s > y1s:
                band_s = bgr[y1s:y2s, :]
                if band_s.size > 0:
                    yield band_s

def upscale_color(img, scale=3):
    """Upscale color image to help OCR."""
    h, w = img.shape[:2]
    return cv2.resize(img, (w*scale, h*scale), interpolation=cv2.INTER_CUBIC)

def best_expiry_from_texts(texts):
    blob = " ".join(t for t in texts if t)
    blob = (blob.replace("O","0").replace("o","0")
                 .replace("S","5").replace("I","1"))
    m = EXPIRY_RE.search(blob)
    if not m:
        return None
    mm, yy = m.groups()
    return f"{int(mm):02d}-{int(yy):02d}"

def read_expiry_color_only(bgr):
    """Try multiple color bands; no grayscale/threshold variants."""
    for band in crop_band_candidates(bgr):
        big = upscale_color(band, scale=3)
        for cfg in EASYOCR_CFGS:
            try:
                texts = reader.readtext(big, **cfg) or []
            except Exception:
                continue
            exp = best_expiry_from_texts(texts)
            if exp:
                return exp
    return None

expiry_result = {}  # fname -> "MM-YY" or "-"

for fname, bgr in images:
    exp = read_expiry_color_only(bgr) or "-"
    expiry_result[fname] = exp
    print(f"{fname} -> Expiry: {expiry_result[fname]}")


1_crop0.jpg -> Expiry: 05-21
2_crop0.jpg -> Expiry: 07-25
3_crop0.jpg -> Expiry: 02-19
4_crop0.jpg -> Expiry: 01-26
5_crop0.jpg -> Expiry: 09-24
